<a href="https://colab.research.google.com/github/andrefcmoraes/simula/blob/main/Gerador_de_G_code_para_Bobinadora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:

import math

def gerar_gcode_avancado():
    # --- Parâmetros Principais ---
    comprimento_cilindro = 300.0  # Comprimento total do tubo (mm)
    diametro_cilindro = 50.0      # Diâmetro externo (mm)
    angulo_bobinagem = 65.0       # Ângulo principal da malha (graus)
    largura_filamento = 6.0       # Largura da fita/tow (mm)
    feedrate_linear = 1500        # Velocidade de avanço base

    # --- Parâmetros de Reforço (Hoop Winding) ---
    voltas_ancoragem = 3          # Quantas voltas a 90 graus em CADA ponta

    # Cálculos Trigonométricos
    angulo_rad = math.radians(angulo_bobinagem)
    passo_linear = (math.pi * diametro_cilindro) / math.tan(angulo_rad)
    circunferencia = math.pi * diametro_cilindro

    # Subdividindo o comprimento do cilindro
    # Espaço ocupado pelas ancoragens nas pontas
    comprimento_hoop = voltas_ancoragem * largura_filamento
    comprimento_helicoidal = comprimento_cilindro - (2 * comprimento_hoop)

    # Quantos passes para fechar a malha?
    passes_totais = int((circunferencia / largura_filamento) * math.sin(angulo_rad))
    if passes_totais < 1: passes_totais = 1

    # Cálculos de Rotação (Eixo Y)
    rotacao_helicoidal = (comprimento_helicoidal / passo_linear) * 360.0
    deslocamento_fase = (largura_filamento / circunferencia) * 360.0

    # Inicialização das Coordenadas
    pos_x = comprimento_hoop # Ponto de partida após a margem esquerda
    pos_y = 0.0

    filename = "bobinagem_pro_com_hoop.gcode"

    with open(filename, 'w') as f:
        # Cabeçalho de Inicialização
        f.write("; --- G-code Filament Winding PRO ---\n")
        f.write("G21 ; Milimetros\n")
        f.write("G90 ; Posicionamento absoluto\n")
        f.write("G28 X ; Homing do Eixo X\n")
        f.write("G92 Y0 X0 ; Zera a posicao atual\n")
        f.write(f"G1 F{feedrate_linear}\n\n")

        # Posiciona no inicio da área helicoidal para o primeiro passe
        f.write(f"G1 X{pos_x:.2f} Y0.00 ; Posicionamento inicial\n\n")

        for passe in range(passes_totais):
            f.write(f"; ====== PASSE {passe + 1} DE {passes_totais} ======\n")

            # 1. Movimento Helicoidal (IDA)
            pos_x = comprimento_cilindro - comprimento_hoop
            pos_y += rotacao_helicoidal
            f.write(f"G1 X{pos_x:.2f} Y{pos_y:.2f} ; Helicoidal Ida\n")

            # 2. Hoop Winding (Extremidade Direita)
            for _ in range(voltas_ancoragem):
                pos_x += largura_filamento
                pos_y += 360.0
                f.write(f"G1 X{pos_x:.2f} Y{pos_y:.2f} ; Hoop Direito (90 graus)\n")

            # 3. Movimento Helicoidal (VOLTA)
            pos_x = comprimento_hoop
            pos_y += rotacao_helicoidal
            f.write(f"G1 X{pos_x:.2f} Y{pos_y:.2f} ; Helicoidal Volta\n")

            # 4. Hoop Winding (Extremidade Esquerda) + Deslocamento de Fase
            for i in range(voltas_ancoragem):
                pos_x -= largura_filamento # Carro recua lentamente
                if i == voltas_ancoragem - 1:
                    # Na última volta de ancoragem, adicionamos a fase para não sobrepor
                    pos_y += 360.0 + deslocamento_fase
                    f.write(f"G1 X{pos_x:.2f} Y{pos_y:.2f} ; Hoop Esquerdo + FASE\n")
                else:
                    pos_y += 360.0
                    f.write(f"G1 X{pos_x:.2f} Y{pos_y:.2f} ; Hoop Esquerdo\n")
            f.write("\n")

        f.write("; --- Fim da Bobinagem ---\n")
        f.write("M84 ; Desliga motores\n")

    print(f"Sucesso! Arquivo '{filename}' gerado.")
    print(f"Zonas: {comprimento_hoop}mm de Hoop | {comprimento_helicoidal}mm de Helicoidal | {comprimento_hoop}mm de Hoop")

if __name__ == "__main__":
    gerar_gcode_avancado()

Sucesso! Arquivo 'bobinagem_pro_com_hoop.gcode' gerado.
Zonas: 18.0mm de Hoop | 264.0mm de Helicoidal | 18.0mm de Hoop
